In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import sys
import math
import numpy as np
import torch
import torch.nn as nn

from torchinfo import summary
sys.path.append('/workspace/gan/3D-FM-GAN')

from models.encoders.helpers import get_blocks, Flatten, bottleneck_IR, bottleneck_IR_SE

from models.networks import FMGAN, EncoderW, EncoderT, GradualStyleEncoder, FMGenerator
from torch.nn import Linear, Conv2d, BatchNorm2d, PReLU, Sequential, Module

from accelerate import Accelerator

accelerator = Accelerator()

In [13]:
class make_empty_object(object):
    pass

args = make_empty_object()
args.wplus_num_layers = 50
args.output_size = 256
args.input_size = 256
args.ckpt = '/workspace/gan/3D-FM-GAN/pretraiend/550000.pt'
fmg = FMGenerator(args)

# accelerator.device: cuda
fmg = accelerator.prepare(fmg)

In [14]:
# fmg.cuda()
fmg.device()

{'stylegan': device(type='cuda', index=0),
 'enc_t': device(type='cuda', index=0),
 'enc_w': device(type='cuda', index=0),
 'enc_wplus': device(type='cuda', index=0)}

In [11]:
result = fmg(torch.randn(3, 3, 256, 256).to('cuda'), torch.randn(3, 3, 256, 256).to('cuda'))

torch.Size([3, 1, 512]) torch.Size([3, 14, 512])


In [12]:
result.shape

torch.Size([3, 3, 256, 256])

In [24]:
styles = nn.ModuleList()
coarse_ind = 3
middle_ind = 7

for i in range(n_styles):
  if i < coarse_ind:
      style = GradualStyleBlock(512, 512, 16)
  elif i < middle_ind:
      style = GradualStyleBlock(512, 512, 32)
  else:
      style = GradualStyleBlock(512, 512, 64)
  styles.append(style)

In [27]:
len(styles)

[autoreload of models.encoders.helpers failed: Traceback (most recent call last):
  File "/opt/conda/lib/python3.8/site-packages/IPython/extensions/autoreload.py", line 245, in check
    superreload(m, reload, self.old_objects)
  File "/opt/conda/lib/python3.8/site-packages/IPython/extensions/autoreload.py", line 410, in superreload
    update_generic(old_obj, new_obj)
  File "/opt/conda/lib/python3.8/site-packages/IPython/extensions/autoreload.py", line 347, in update_generic
    update(a, b)
  File "/opt/conda/lib/python3.8/site-packages/IPython/extensions/autoreload.py", line 317, in update_class
    update_instances(old, new)
  File "/opt/conda/lib/python3.8/site-packages/IPython/extensions/autoreload.py", line 280, in update_instances
    ref.__class__ = new
TypeError: __class__ assignment: 'Bottleneck' object layout differs from 'Bottleneck'
]


14

In [3]:
enc_wplus = GradualStyleEncoder(50)

In [4]:
enc_wplus(torch.randn(2, 3, 256, 256)).shape

torch.Size([2, 14, 512])

In [9]:
modulelist = enc_wplus.body._modules.values()
for i, l in enumerate(modulelist):
    print(i, l)

0 bottleneck_IR(
  (shortcut_layer): MaxPool2d(kernel_size=1, stride=2, padding=0, dilation=1, ceil_mode=False)
  (res_layer): Sequential(
    (0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (2): PReLU(num_parameters=64)
    (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
)
1 bottleneck_IR(
  (shortcut_layer): MaxPool2d(kernel_size=1, stride=1, padding=0, dilation=1, ceil_mode=False)
  (res_layer): Sequential(
    (0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (2): PReLU(num_parameters=64)
    (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (4): BatchNorm2d(64, eps=1e-0

In [ ]:
class GradualStyleEncodaer(nn.Module):
    def __init__(self, num_layers, mode='ir', output_size=256, input_nc=3):
        super().__init__()
        if mode == 'ir':
            unit_module = bottleneck_IR
        elif mode == 'ir_se':
            unit_module = bottleneck_IR_SE

        

        self.input_layer = Sequential(Conv2d(input_nc, 64, (3, 3), 1, 1, bias=False),
                                      BatchNorm2d(64),
                                      PReLU(64))

        n_styles = int(math.log(output_size, 2)) * 2 - 2
        self.styles = nn.ModuleList()
        self.style_count = n_styles
        self.coarse_ind = 3
        self.middle_ind = 7
        for i in range(self.style_count):
            if i < self.coarse_ind:
                style = GradualStyleBlock(512, 512, 16)
            elif i < self.middle_ind:
                style = GradualStyleBlock(512, 512, 32)
            else:
                style = GradualStyleBlock(512, 512, 64)
            self.styles.append(style)

            
        self.latlayer1 = nn.Conv2d(256, 512, kernel_size=1, stride=1, padding=0)
        self.latlayer2 = nn.Conv2d(128, 512, kernel_size=1, stride=1, padding=0)

    def forward(self, x):
        x = self.input_layer(x)

        latents = []
        

In [46]:
enc_w = EncoderW()

In [47]:
enc_w(torch.randn(2, 3, 256, 256)).shape

torch.Size([2, 512])

In [49]:
enc_t = EncoderT()

In [50]:
enc_t(torch.randn(2, 3, 256, 256)).shape

torch.Size([2, 512, 4, 4])

In [6]:
import math
n_styles = int(math.log(1024, 2)) * 2 - 2
n_styles

18

In [22]:
from torchvision.models import resnet18

resnet = resnet18(pretrained=False)

In [37]:
test = torch.nn.Conv2d(512, 512, kernel_size=3, padding=1, stride=2)

In [39]:
test(enc_t(torch.randn(2, 3, 256, 256))).shape

torch.Size([2, 512, 4, 4])

In [42]:
enc_w = Encoder('modulation')
enc_w(torch.randn(2, 3, 256, 256)).shape

torch.Size([2, 512, 1, 1])

In [27]:
batch_size = 16
summary(resnet, input_size=(batch_size, 3, 128, 128))

Layer (type:depth-idx)                   Output Shape              Param #
ResNet                                   [16, 1000]                --
├─Conv2d: 1-1                            [16, 64, 64, 64]          9,408
├─BatchNorm2d: 1-2                       [16, 64, 64, 64]          128
├─ReLU: 1-3                              [16, 64, 64, 64]          --
├─MaxPool2d: 1-4                         [16, 64, 32, 32]          --
├─Sequential: 1-5                        [16, 64, 32, 32]          --
│    └─BasicBlock: 2-1                   [16, 64, 32, 32]          --
│    │    └─Conv2d: 3-1                  [16, 64, 32, 32]          36,864
│    │    └─BatchNorm2d: 3-2             [16, 64, 32, 32]          128
│    │    └─ReLU: 3-3                    [16, 64, 32, 32]          --
│    │    └─Conv2d: 3-4                  [16, 64, 32, 32]          36,864
│    │    └─BatchNorm2d: 3-5             [16, 64, 32, 32]          128
│    │    └─ReLU: 3-6                    [16, 64, 32, 32]          --
│

In [ ]:
class Encoder(nn.Module):
    def __init__(self, emb_space):
        super(Encoder, self).__init__()
        resnet = resnet18(pretrained=False)

        if emb_space == 'tensor':
            self.features = nn.Sequential(*list(resnet.children())[:-2])
        elif emb_space == 'modulation':
            self.features = nn.Sequential(*list(resnet.children())[:-1])
        
    def forward(self, x):
        x = self.features(x)
        return x